# Tasks
The objective of this notebook is to use machine learning model to provide a ward national landcover in 2021. 

In [1]:
# import libraries
import os
import ee
import geemap

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWjmrwXjDNR9lw1voWMPS_dZCWQtFMhh3p9jNL6FQSJFY93gBOzvAlc

Successfully saved authorization token.


In [3]:
!pwd

/Users/noe/Documents/PostDoc/Workshop/Day 3


## Select the ward of interest and load it raster

In [4]:
Map = geemap.Map(center=(-33.918861, 18.423300))
# Upload the shapefile of the ROI(SA)
w    = geemap.shp_to_ee('/Users/noe/Documents/PostDoc/Workshop/Day 3/SA4/ADM4_PCODE_ZA5950089.shp')
Map.addLayer(w, {}, 'ward_of_interest')
Map

Map(center=[-33.918861, 18.4233], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [5]:
# Get Sentinel-2 data
s2 = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2021-01-01','2021-12-31').filterBounds(w).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',70)).filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT',70))
rgbVis  = {
  'min': 0.0,
  'max': 0.3,
  'gamma':1.4,
  'bands': ['B4_median', 'B3_median', 'B2_median'],
}
# ///////////////// Use Median Band in parameters///////////////////////

median = s2.reduce(ee.Reducer.median())

# ////////////////Mosaic these two image collections///////////////////

mosaic_1 = ee.ImageCollection.fromImages([median]).mosaic()

# ////////////////////////Clip by assets or roi////////////////////////

landsat_8_2021 = mosaic_1.clip(w)

# /////////////// Display the clipped ImageCollection/////////////////

Map.addLayer(landsat_8_2021, rgbVis, 'sentinel_roi')
Map

Map(bottom=2759.0, center=[-33.918861, 18.4233], controls=(WidgetControl(options=['position', 'transparent_bg'…

In [6]:
# scale the data
def scaleData(img):
    prop = img.toDictionary()
    myImg = img.select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']).divide(10000)
    myImg = myImg.addBands(img.select(['QA60'])).set(prop).copyProperties(img,['system:time_start','system:footprint'])
    return ee.Image(myImg)